# Case Study Answers

## Time Series Analysis

### The sales at ABC have been like a rollercoaster, going up and down every month. Can you chart this fluctuating journey and identify any seasonal patterns that might explain these ups and downs?

In [57]:
import plotly.express as px 
import pandas as pd

df = pd.read_csv("./Files/Sales_Data.csv")

df['OrderDate'] = pd.to_datetime(df['OrderDate']) 
df['YearMonth'] = df['OrderDate'].dt.to_period('M') 
df['YearMonth'] = df['YearMonth'].astype(str)

last_year_data = df[df['OrderDate'] >= '2022-01-01'] 
last_year_monthly_sales = last_year_data.groupby('YearMonth')['TotalAmount'].sum().reset_index()

fig = px.line(last_year_monthly_sales, x='YearMonth', y='TotalAmount', title='Monthly Sales Trend for the Past Year')
fig.update_traces(mode='lines+markers+text', texttemplate='%{y:.2s}k', textposition='top center', textfont_color='black')
fig.update_xaxes(title_text='Month')
fig.update_yaxes(title_text='Total Sales Amount')
fig.show()

### It's a busy week at ABC, but is every day equally busy? Your task is to find out if weekdays bring in more sales than weekends or vice versa. This could help the company in planning their staffing and marketing strategies.

In [55]:
# Extract the day of the week from 'OrderDate'
df['Weekday'] = df['OrderDate'].dt.day_name()

weekday_sales = df.groupby('Weekday')['TotalAmount'].sum().reset_index()

weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
# Convert the day of the week column to a categorical column with the specified order
weekday_sales['Weekday'] = pd.Categorical(weekday_sales['Weekday'], categories=weekday_order, ordered=True)
weekday_sales = weekday_sales.sort_values('Weekday')

# Plotting the weekday sales using plotly express
# Extract the day of the week from 'OrderDate'
df['Weekday'] = df['OrderDate'].dt.day_name()

weekday_sales = df.groupby('Weekday')['TotalAmount'].sum().reset_index()

weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
# Convert the day of the week column to a categorical column with the specified order
weekday_sales['Weekday'] = pd.Categorical(weekday_sales['Weekday'], categories=weekday_order, ordered=True)
weekday_sales = weekday_sales.sort_values('Weekday')

# Plotting the weekday sales using plotly express
fig = px.bar(weekday_sales, x='Weekday', y='TotalAmount', color='Weekday',
             title='Sales by Weekday', labels={'Weekday': 'Weekday', 'TotalAmount': 'Total Sales Amount'})
fig.update_traces(texttemplate='%{y:.2s}', textposition='auto', textfont_color='white')
fig.show()


## Customer Segmentation

### ABC wants to roll out a VIP program but isn't sure who to invite. Your task is to identify the top 10% of customers based on their spending. What makes these customers special?

In [37]:
# Aggregate sales data by customer
customer_sales = df.groupby('CustomerID')['TotalAmount'].sum().reset_index()

# Calculate the 90th percentile of customer spending
top_10_percentile_value = customer_sales['TotalAmount'].quantile(0.9)

# Identify the top 10% of customers based on their spending
top_10_percent_customers = customer_sales[customer_sales['TotalAmount'] >= top_10_percentile_value]

# Display the 'CustomerID' and 'TotalAmount' for the top 10% of customers
top_10_percent_customers[['CustomerID', 'TotalAmount']].head(10)

,CustomerID,TotalAmount
1,2,6352.64
3,4,9515.14
7,9,8867.89
8,10,7464.11
10,12,6130.85
12,14,6631.16
17,21,7048.61
34,40,8046.50
76,88,6770.81
82,97,8030.53


## Product Analysis

### In the vast inventory of ABC, some products are stars while others are not. Can you spotlight the top 5 best-selling products in each category and suggest why they might be the customer favorites?

In [38]:
# Aggregate sales data by product and category
product_sales_by_category = df.groupby(['Category', 'ProductName'])['Quantity'].sum().reset_index()

# Sort the data to get the top 5 best-selling products in each category
top_5_products_by_category = product_sales_by_category.sort_values(['Category', 'Quantity'], ascending=[True, False])
top_5_products_by_category = top_5_products_by_category.groupby('Category').head(5).reset_index(drop=True)

# Display the top 5 best-selling products in each category
top_5_products_by_category

,Category,ProductName,Quantity
0,Electronics,Product_297,18
1,Electronics,Product_309,16
2,Electronics,Product_326,16
3,Electronics,Product_295,15
4,Electronics,Product_284,14
5,Fashion,Product_228,14
6,Fashion,Product_390,14
7,Fashion,Product_148,12
8,Fashion,Product_248,12
9,Fashion,Product_174,11


###	Returns are a headache for any retail business. ABC is no exception. Your challenge is to identify products that are frequently returned and hypothesize why this might be happening.

In [39]:
# Filter data for returned orders
returned_orders = df[df['IsReturned'] == 1]

# Count the number of times each product is returned
returned_products_count = returned_orders.groupby('ProductName')['IsReturned'].count().reset_index()

# Sort the data to get the most frequently returned products
most_returned_products = returned_products_count.sort_values('IsReturned', ascending=False).head(10)

# Display the most frequently returned products
most_returned_products

,ProductName,IsReturned
279,Product_489,6
142,Product_297,6
57,Product_183,5
208,Product_392,5
206,Product_390,5
109,Product_248,5
204,Product_388,4
64,Product_193,4
73,Product_205,4
163,Product_326,4


## Payment and Returns

### People have different preferences when it comes to parting with their money. Can you find out if there's a preferred payment method for higher-value orders? This could influence future payment options offered by ABC.

In [50]:

# Calculate the median order value for each payment method
median_order_value_by_payment = df.groupby('PaymentMethod')['TotalAmount'].median().reset_index()

# Sort the data by median order value
median_order_value_by_payment = median_order_value_by_payment.sort_values('TotalAmount', ascending=False)

# Plotting the median order value by payment method using plotly express
fig = px.bar(median_order_value_by_payment, x='PaymentMethod', y='TotalAmount', 
             title='Median Order Value by Payment Method', labels={'PaymentMethod': 'Payment Method', 'TotalAmount': 'Median Order Value'}, color='PaymentMethod')
fig.update_traces(texttemplate='%{y:.2f}', textposition='auto', textfont_color='white')
fig.show()

### Returns are like a leak in a boat, and ABC wants to plug it. Are there specific categories or payment methods that are more prone to returns? Your findings could help the company tighten its return policy.

In [41]:
# Calculate the return rate for each category
total_orders_by_category = df.groupby('Category')['OrderID'].count().reset_index()
returned_orders_by_category = returned_orders.groupby('Category')['OrderID'].count().reset_index()
return_rate_by_category = pd.merge(total_orders_by_category, returned_orders_by_category, on='Category', how='left')
return_rate_by_category.columns = ['Category', 'TotalOrders', 'ReturnedOrders']
return_rate_by_category['ReturnRate'] = (return_rate_by_category['ReturnedOrders'] / return_rate_by_category['TotalOrders']) * 100

# Calculate the return rate for each payment method
total_orders_by_payment = df.groupby('PaymentMethod')['OrderID'].count().reset_index()
returned_orders_by_payment = returned_orders.groupby('PaymentMethod')['OrderID'].count().reset_index()
return_rate_by_payment = pd.merge(total_orders_by_payment, returned_orders_by_payment, on='PaymentMethod', how='left')
return_rate_by_payment.columns = ['PaymentMethod', 'TotalOrders', 'ReturnedOrders']
return_rate_by_payment['ReturnRate'] = (return_rate_by_payment['ReturnedOrders'] / return_rate_by_payment['TotalOrders']) * 100

# Display the return rates by category and payment method
return_rate_by_category, return_rate_by_payment

(        Category  TotalOrders  ReturnedOrders  ReturnRate
 0    Electronics          336             172   51.190476
 1        Fashion          343             179   52.186589
 2  Home & Garden          321             165   51.401869,
   PaymentMethod  TotalOrders  ReturnedOrders  ReturnRate
 0          Cash          311             176   56.591640
 1   Credit Card          343             170   49.562682
 2        PayPal          346             170   49.132948)

In [49]:
# Plot return rates by category
import plotly.graph_objects as go
import plotly.express as px

# Define a color sequence for categories
colors_category = px.colors.qualitative.Plotly

# Plot return rates by category
fig1 = go.Figure(data=[
    go.Bar(
        x=return_rate_by_category['Category'], 
        y=return_rate_by_category['ReturnRate'], 
        marker_color=colors_category[:len(return_rate_by_category)],  # Use the first n colors
        text=return_rate_by_category['ReturnRate'].round(2),
        textposition='auto',
        textfont_color='white'
    )
])
fig1.update_layout(title_text='Return Rate by Category')
fig1.show()

# Define a color sequence for payment methods
colors_payment = px.colors.qualitative.Vivid

# Plot return rates by payment method
fig2 = go.Figure(data=[
    go.Bar(
        x=return_rate_by_payment['PaymentMethod'], 
        y=return_rate_by_payment['ReturnRate'], 
        marker_color=colors_payment[:len(return_rate_by_payment)],  # Use the first n colors
        text=return_rate_by_payment['ReturnRate'].round(2),
        textposition='auto',
        textfont_color='white'
    )
])
fig2.update_layout(title_text='Return Rate by Payment Method')
fig2.show()